# Introduction
This file converts the cleaned raw dataset into a single merged file that the TFTModel can work on. The script version available at [prepare_data.py](../script/prepare_data.py).

If you need to change the input feature set, only add that info in the `"data"` section of the json configuration  file. This notebook will update the rest (at least feature column mappings and locations) . If you have pivoted dynamic feature and need to melt that date columns, make sure to keep the feature name as `string` in `"dynamic_features_map"`. If it is already melted and your dynamic file has a `Date` column, `list` or `string` format both is fine.

In the final output all null values are replaced with 0. If you don't want that, comment that out.

# Import libraries

In [1]:
import argparse
import sys
sys.path.append( '..' )
from Class.DataMerger import *

# Setup storage

## Googe drive
Not needed. But set `running_on_colab = True` if using.

In [ ]:
running_on_colab = False

if running_on_colab:
    from google.colab import drive
    drive.mount('/content/drive')

    %cd /content/drive/My Drive/COVID-19-forecast/src/notebooks

## Input
If running on colab, modify the below paths accordingly. Note that this config.json is different from the config.json in TF2 folder as that is for the old dataset.

In [8]:
from dataclasses import dataclass

@dataclass
class args:
    # folder where the cleaned feature file are at
    # dataPath = '../../dataset_raw/CovidDecember12-2021'
    dataPath = '../../dataset_raw/CovidMay17-2022'


    supportPath = '../../dataset_raw/Support files'
    outputPath = '../2022_May/'
    configPath = '../config_2022_May.json'

In [9]:
# create output path if it doesn't exist
if not os.path.exists(args.outputPath):
    print(f'Creating output directory {args.outputPath}')
    os.makedirs(args.outputPath, exist_ok=True)

# load config file
with open(args.configPath) as inputFile:
    config = json.load(inputFile)
    print(f'Config file loaded from {args.configPath}')
    inputFile.close()

Config file loaded from ../config_2022_May.json


# Data merger

## Total features

In [ ]:
# get merger class
dataMerger = DataMerger(config, args.dataPath, args.supportPath)
total_df = dataMerger.get_all_features()

In [ ]:
output_path_total = os.path.join(args.outputPath, 'Total.csv') 
print(f'Writing total data to {output_path_total}\n')
total_df.round(4).to_csv(output_path_total, index=False)

In [10]:
print('Updating config file')
dataMerger.update_config(args.configPath)

Unique counties present 3142
Merging feature Age Distribution.csv with length 3142
Merging feature Air Pollution.csv with length 3142
Merging feature Health Disparities.csv with length 3142

Merged static features have 3142 counties
   FIPS  AgeDist  AirPollution  HealthDisp
0  1001   0.5017        0.5210      0.2606
1  1003   0.6095        0.4371      0.2039
2  1005   0.5797        0.5090      0.6562
3  1007   0.5427        0.4910      0.5320
4  1009   0.5755        0.5210      0.4462
Reading Disease Spread.csv
Min date 2020-02-28 00:00:00, max date 2022-05-17 00:00:00
Length 2545020.
Reading Transmissible Cases.csv
Min date 2020-02-28 00:00:00, max date 2022-05-17 00:00:00
Length 2545020.
Reading Vaccination.csv
Min date 2020-12-13 00:00:00, max date 2022-05-17 00:00:00
Length 1679704.
Reading Social Distancing.csv
Min date 2020-02-28 00:00:00, max date 2022-05-17 00:00:00
Length 2545020.
Total dynamic feature shape (2587742, 7)
   FIPS              Name       Date  DiseaseSpread  Tr

## Rurality cut

In [11]:
# can be used as cache to perform different rurality or population cuts
# total_df = pd.read_csv(output_path_total)

# you can define Rurality cut in 'data'->'support'
# Rurality cut has to be set true. and also set lower and upper limit in RuralityRange and/or MADRange
# having -1 in either of these two will result in ignoring that key
if dataMerger.need_rurality_cut():
    rurality_df = dataMerger.rurality_cut(total_df)

    output_path_rurality_cut = os.path.join(args.outputPath, 'Rurality_cut.csv')
    print(f'Writing rurality cut data to {output_path_rurality_cut}\n')
    rurality_df.round(4).to_csv(output_path_rurality_cut, index=False)

Slicing based on top 100 counties by population
Writing population cut data to ../2022_May/Population_cut.csv



## Population cut

In [ ]:
# you can define 'Population cut' in 'data'->'support'
# this means how many of top counties you want to keep
if dataMerger.need_population_cut():
    top_df = dataMerger.population_cut(total_df)

    output_path_population_cut = os.path.join(args.outputPath, 'Population_cut.csv')
    print(f'Writing population cut data to {output_path_population_cut}\n')
    top_df.round(4).to_csv(output_path_population_cut, index=False)

# Run TFT model

Now you can use the [TFT_on_top_500_counties.ipynb](/Source/notebooks/TFT_on_top_500_counties.ipynb) notebook to run experiment on top 500 counties.